### 1. Przygotowanie danych

In [ ]:
import pandas as pd

# Ścieżki do plików CSV
inventory_path = "data/inventory.csv"
workcenters_path = "data/workcenters.csv"
workcenter_availability_path = "data/workcenter_availability.csv"
routes_path = "data/routes.csv"
customers_path = "data/customers.csv"
sales_orders_path = "data/sales_orders.csv"
bom_path = "data/bom.csv"

# Import danych
inventory = pd.read_csv(inventory_path)
workcenters = pd.read_csv(workcenters_path)
workcenter_availability = pd.read_csv(workcenter_availability_path)
routes = pd.read_csv(routes_path)
customers = pd.read_csv(customers_path)
sales_orders = pd.read_csv(sales_orders_path)
bom = pd.read_csv(bom_path)

# Podgląd danych
print("Inventory data preview:")
print(inventory.head())

print("\nWorkcenters data preview:")
print(workcenters.head())

print("\nWorkcenter availability data preview:")
print(workcenter_availability.head())

print("\nRoute data preview:")
print(routes.head())

print("\nCustomers data preview:")
print(customers.head())

print("\nSales orders data preview:")
print(sales_orders.head())

print("\nBom data preview:")
print(bom.head())


In [ ]:
# Weryfikacja braków danych
def check_missing(dataframes):
    for name, df in dataframes.items():
        print(f"Missing values in {name}:")
        print(df.isnull().sum())
        print()

datasets = {
    "inventory": inventory,
    "workcenters": workcenters,
    "workcenter_availability": workcenter_availability,
    "routes": routes,
    "customers": customers,
    "sales_orders": sales_orders,
    "bom": bom,
}

check_missing(datasets)


In [ ]:
# Sprawdzenie typów danych
def check_data_types(dataframes):
    for name, df in dataframes.items():
        print(f"Data types in {name}:")
        print(df.dtypes)
        print()

check_data_types(datasets)

In [ ]:
# Konwersje typów danych
workcenter_availability['Date'] = pd.to_datetime(workcenter_availability['Date'])
sales_orders['Delivery Date'] = pd.to_datetime(sales_orders['Delivery Date'])

#### Tworzenie drzewa BOM i obliczenia


In [21]:
def build_complete_bom(bom_df, item_id, fg_id, level=0):
    """
    Rekursywnie tworzy pełną strukturę BOM dla danego FG od poziomu 0 do samego dołu.
    """
    tree = []
    components = bom_df[bom_df["BOM ID"] == item_id]

    for _, row in components.iterrows():
        # Dodanie rekordu do drzewa
        tree.append({
            "FG Item ID": fg_id,          # Wyrob gotowy, dla którego tworzymy BOM
            "Parent ID": item_id,         # Bezpośredni rodzic komponentu
            "Component ID": row["Component ID"],  # Komponent w BOM
            "Level": level + 1,           # Poziom w hierarchii
            "Quantity": row["Quantity per Unit"]   # Ilość wymagana w BOM
        })
        # Rekurencja dla kolejnych poziomów komponentu
        subtree = build_complete_bom(bom_df, row["Component ID"], fg_id, level + 1)
        tree.extend(subtree)

    return tree

def generate_full_bom_structure(inventory_df, bom_df):
    """
    Generuje pełną strukturę BOM dla wszystkich wyrobów gotowych (FG).
    """
    fg_items = inventory_df[inventory_df["Item Type"] == "FG"]["Item ID"]
    bom_structure = []

    for fg_item in fg_items:
        # Rekurencyjne budowanie drzewa BOM dla każdego FG
        subtree = build_complete_bom(bom_df, fg_item, fg_item)
        bom_structure.extend(subtree)

    return pd.DataFrame(bom_structure)

# Generowanie pełnej struktury BOM
full_bom_structure_df = generate_full_bom_structure(inventory, bom)

# Podgląd danych
print("Full BOM Structure DataFrame:")
print(full_bom_structure_df.head())

# Zapisanie do pliku CSV (opcjonalne)
full_bom_structure_df.to_csv("prepared_data/full_bom_structure.csv", index=False)


Full BOM Structure DataFrame:
         FG Item ID          Parent ID       Component ID  Level  Quantity
0  INV-STRC-FG-0001   INV-STRC-FG-0001  INV-MODL-SFG-1410      1         1
1  INV-STRC-FG-0001  INV-MODL-SFG-1410     INV-RM-RM-0017      2         2
2  INV-STRC-FG-0001  INV-MODL-SFG-1410     INV-RM-RM-0063      2         1
3  INV-STRC-FG-0001  INV-MODL-SFG-1410     INV-RM-RM-0091      2         2
4  INV-STRC-FG-0001  INV-MODL-SFG-1410     INV-RM-RM-0094      2         3
